In [1]:
import openai
import wandb
import pandas as pd
from math import exp
import re
from IPython.display import display, HTML
import os

from dotenv import find_dotenv, load_dotenv
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
assert os.getenv("OPENAI_API_KEY"), "No OPENAI_API_KEY defined in .env."

True

In [3]:
!wandb online

W&B online. Running your script from this directory will now sync to the cloud.


In [4]:
project_name = "GPT-3 blog title"
run = wandb.init(project=project_name, job_type='prompting')

wandb: Currently logged in as: benneo. Use `wandb login --relogin` to force relogin


In [5]:
# choose a fine-tuned model
artifact = run.use_artifact(
    f"benneo/{project_name}/fine_tune_details:v1", type="fine_tune_details"
)

# load the fine-tuned model
fine_tuned_model = artifact.metadata['fine_tuned_model']
fine_tuned_model

'babbage:ft-personal:blog-title-scorer-2022-12-01-04-05-26'

In [7]:
def get_pred(res):
    pred = res["choices"][0]["text"]
    class_prob = exp(res["choices"][0]["logprobs"]["token_logprobs"][0])

    if pred == " bad":
        class_prob = 1.0 - class_prob

    return pred, class_prob

In [11]:
base_prompt = "Rewrite the following blog post title into six different titles but optimized for social media virality: {0}\n\n-"

title_input = "Using GPT-3 to Optimize My Medium Blog Titles"

r = openai.Completion.create(
    model="text-davinci-003",
    prompt=base_prompt.format(title_input),
    temperature=0,  # deterministic output; should set to 0.7 or 1 elsewise
    max_tokens=256,  # fine for small titles but may need to bump
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

gen_titles = re.split(r" ?\n-", r["choices"][0]["text"])
gen_titles

['1. "Unlock the Power of GPT-3 to Boost Your Medium Blog Titles!"',
 '2. "GPT-3: The Secret to Crafting Viral Medium Blog Titles!"',
 '3. "GPT-3: The Key to Crafting the Perfect Medium Blog Title!"',
 '4. "GPT-3: Unlock the Potential of Your Medium Blog Titles!"',
 '5. "GPT-3: Unlock the Magic of Your Medium Blog Titles!"',
 '6. "GPT-3: Unlock the Power of Your Medium Blog Titles!"']

In [17]:
base_prompt = "Rewrite the following blog post title into 10 different titles but optimized for social media virality and quality: {0}\n\n-"

title_input = "Using GPT-3 to Optimize My Medium Blog Titles"

r = openai.Completion.create(
    model="text-davinci-003",
    prompt=base_prompt.format(title_input),
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

gen_titles = re.split(r" ?\n-", r["choices"][0]["text"])
gen_titles

['1. Unlocking the Power of GPT-3 to Supercharge Your Medium Blog Titles',
 '2. Harnessing GPT-3 to Take Your Medium Blog Titles to the Next Level',
 '3. GPT-3: The Secret Weapon for Crafting Killer Medium Blog Titles',
 '4. GPT-3: The Key to Crafting Irresistible Medium Blog Titles',
 '5. GPT-3: The Secret Sauce for Crafting Magnetic Medium Blog Titles',
 '6. GPT-3: The Magic Ingredient for Crafting Unforgettable Medium Blog Titles',
 '7. GPT-3: The Secret Tool for Crafting Engaging Medium Blog Titles',
 '8. GPT-3: The Secret Tool for Crafting Irresistible Medium Blog Titles',
 '9. GPT-3: The Key to Crafting Magnetic Medium Blog Titles',
 '10. GPT-3: The Secret to Crafting Unforgettable Medium Blog Titles']

In [18]:
base_prompt = "Rewrite the following blog post title into 10 different titles but optimized for virality and quality: {0}\n\n-"

title_input = "Using GPT-3 to Optimize My Medium Blog Titles"

r = openai.Completion.create(
    model="text-davinci-003",
    prompt=base_prompt.format(title_input),
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

gen_titles = re.split(r" ?\n-", r["choices"][0]["text"])
gen_titles

[' 10 Ways GPT-3 Can Help You Optimize Your Medium Blog Titles',
 ' Unlocking the Power of GPT-3 to Optimize Your Medium Blog Titles',
 ' How GPT-3 Can Help You Create the Perfect Medium Blog Title',
 ' GPT-3: The Secret to Crafting the Perfect Medium Blog Title',
 ' GPT-3: The Key to Crafting the Perfect Medium Blog Title',
 ' GPT-3: Unlocking the Power to Optimize Your Medium Blog Titles',
 ' GPT-3: The Secret to Optimizing Your Medium Blog Titles',
 ' GPT-3: The Key to Optimizing Your Medium Blog Titles',
 ' GPT-3: The Ultimate Guide to Optimizing Your Medium Blog Titles',
 ' GPT-3: The Definitive Guide to Optimizing Your Medium Blog Titles']

combining the fine tuned scorer model and prompting model into one function

In [67]:
def title_optimizer(title_input, temp=1.0):
    base_prompt = "Rewrite the following blog post title into 6 different titles but optimized for virality and quality: {0}\n\n-"
    finetune_prompt = "Title: {0} ->"

    r = openai.Completion.create(
        model="text-davinci-003",
        prompt=base_prompt.format(title_input),
        temperature=0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    gen_titles = re.split(r" ?\n-", r["choices"][0]["text"])
    gen_titles = list(set(gen_titles + [title_input]))

    ranked_titles = []

    for gen_title in gen_titles:
        r = openai.Completion.create(
            model=fine_tuned_model,
            prompt=finetune_prompt.format(gen_title),
            temperature=0,
            max_tokens=1,
            logprobs=1,
        )

        _, class_prob = get_pred(r)

        ranked_titles.append(
            (
                f"{gen_title}" if gen_title == title_input else gen_title,
                class_prob,
            )
        )

    df = pd.DataFrame(ranked_titles, columns=["Title", "Good Prob"])
    df = df.sort_values(by="Good Prob", ascending=False)

    # return a html output of the data frame and color the text if it matches the input and remove index and convert good prob to %

    df_styled = (
        df.style.hide(axis="index")
        .applymap(
            lambda x: "color: #00FF00" if x == title_input else "",
            subset=["Title"],
        )
        .format({"Good Prob": "{:.1%}"})
    )

    display(df_styled)

    return df, df_styled


out, out_html = title_optimizer("Using GPT-3 to Optimize My Medium Blog Titles", temp=0)

Title,Good Prob
GPT-3: The Definitive Guide to Optimizing Your Medium Blog Titles,76.8%
GPT-3: The Ultimate Guide to Optimizing Your Medium Blog Titles,74.4%
10 Ways GPT-3 Can Help You Optimize Your Medium Blog Titles,64.0%
GPT-3: The Secret to Optimizing Your Medium Blog Titles,63.3%
GPT-3: The Key to Optimizing Your Medium Blog Titles,61.2%
Unlocking the Power of GPT-3 to Optimize Your Medium Blog Titles,57.3%
Using GPT-3 to Optimize My Medium Blog Titles,56.4%
GPT-3: Unlocking the Power to Optimize Your Medium Blog Titles,56.4%
GPT-3: The Secret to Crafting the Perfect Medium Blog Title,47.8%
How GPT-3 Can Help You Create the Perfect Medium Blog Title,46.8%


testing the function

In [68]:
out

,Title,Good Prob
10,GPT-3: The Definitive Guide to Optimizing You...,0.768009
5,GPT-3: The Ultimate Guide to Optimizing Your ...,0.743880
0,10 Ways GPT-3 Can Help You Optimize Your Medi...,0.640423
4,GPT-3: The Secret to Optimizing Your Medium B...,0.633265
2,GPT-3: The Key to Optimizing Your Medium Blog...,0.611543
8,Unlocking the Power of GPT-3 to Optimize Your...,0.572864
7,Using GPT-3 to Optimize My Medium Blog Titles,0.564085
3,GPT-3: Unlocking the Power to Optimize Your M...,0.564066
6,GPT-3: The Secret to Crafting the Perfect Med...,0.477515
9,How GPT-3 Can Help You Create the Perfect Med...,0.467680


In [69]:
out_html

Title,Good Prob
GPT-3: The Definitive Guide to Optimizing Your Medium Blog Titles,76.8%
GPT-3: The Ultimate Guide to Optimizing Your Medium Blog Titles,74.4%
10 Ways GPT-3 Can Help You Optimize Your Medium Blog Titles,64.0%
GPT-3: The Secret to Optimizing Your Medium Blog Titles,63.3%
GPT-3: The Key to Optimizing Your Medium Blog Titles,61.2%
Unlocking the Power of GPT-3 to Optimize Your Medium Blog Titles,57.3%
Using GPT-3 to Optimize My Medium Blog Titles,56.4%
GPT-3: Unlocking the Power to Optimize Your Medium Blog Titles,56.4%
GPT-3: The Secret to Crafting the Perfect Medium Blog Title,47.8%
How GPT-3 Can Help You Create the Perfect Medium Blog Title,46.8%


In [ ]:
prediction_table = wandb.Table(columns=['Title', 'Good Prob'], data=out)

In [52]:
wandb.log({'predictions': prediction_table})

In [53]:
wandb.finish()

In [16]:
def title_optimizer(fine_tuned_model, title_input, temp=1.0):

    base_prompt = "Rewrite the following blog post title into 6 different titles but optimized for virality and quality: {0}\n\n-"
    finetune_prompt = "Title: {0} ->"

    r = openai.Completion.create(
        model="text-davinci-003",
        prompt=base_prompt.format(title_input),
        temperature=temp,  # 0 is deterministic output; should set to 0.7 or 1 elsewise
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    gen_titles = re.split(r" ?\n-", r["choices"][0]["text"])

    gen_titles = list(set(gen_titles + [title_input]))

    print(gen_titles)

    ranked_titles = []

    for gen_title in gen_titles:
        r = openai.Completion.create(
            model=fine_tuned_model,
            prompt=finetune_prompt.format(gen_title),
            temperature=0,
            max_tokens=1,
            logprobs=1,
        )

        _, class_prob = get_pred(r)

        ranked_titles.append(
            (
                f"{gen_title}" if gen_title == title_input else gen_title,
                class_prob,
            )
        )

    print(ranked_titles)

    df = pd.DataFrame(ranked_titles, columns=["Title", "Good Prob"])
    df = df.sort_values(by="Good Prob", ascending=False)

    # highlight the input text
    df_styled = (
        df.style.hide(axis="index")
        .applymap(
            lambda x: "color: #00FF00" if x == title_input else "",
            subset=["Title"],
        )
        .format({"Good Prob": "{:.2%}"})
    )

    return df_styled


out_styled = title_optimizer('babbage:ft-personal:blog-title-scorer-2022-12-01-04-05-26'
, "Using GPT-3 to Optimize My Medium Blog Titles", temp=0)

[' "GPT-3: Unlock the Potential of Your Medium Blog Titles"', ' "GPT-3: The Tool to Create the Most Viral Medium Blog Titles"', 'Using GPT-3 to Optimize My Medium Blog Titles', ' "GPT-3: The Secret to Crafting the Perfect Medium Blog Title"', ' "Unlock the Power of GPT-3 to Maximize Your Medium Blog Titles"', ' "GPT-3: The Tool to Unlock the Potential of Your Medium Blog Titles"', ' "GPT-3: The Key to Crafting the Most Engaging Medium Blog Titles"']
[(' "GPT-3: Unlock the Potential of Your Medium Blog Titles"', 0.4930236026373763), (' "GPT-3: The Tool to Create the Most Viral Medium Blog Titles"', 0.4205222140099729), ('Using GPT-3 to Optimize My Medium Blog Titles', 0.564084819278315), (' "GPT-3: The Secret to Crafting the Perfect Medium Blog Title"', 0.4684280400212243), (' "Unlock the Power of GPT-3 to Maximize Your Medium Blog Titles"', 0.5690718720017496), (' "GPT-3: The Tool to Unlock the Potential of Your Medium Blog Titles"', 0.4786454237017592), (' "GPT-3: The Key to Crafting 

In [17]:
out_styled

Title,Good Prob
"""Unlock the Power of GPT-3 to Maximize Your Medium Blog Titles""",56.91%
Using GPT-3 to Optimize My Medium Blog Titles,56.41%
"""GPT-3: The Key to Crafting the Most Engaging Medium Blog Titles""",50.47%
"""GPT-3: Unlock the Potential of Your Medium Blog Titles""",49.30%
"""GPT-3: The Tool to Unlock the Potential of Your Medium Blog Titles""",47.86%
"""GPT-3: The Secret to Crafting the Perfect Medium Blog Title""",46.84%
"""GPT-3: The Tool to Create the Most Viral Medium Blog Titles""",42.05%
